In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the oxford_flowers102 Dataset
# ---------------------------------------------------
# Create train/validation split
(dataset_train, dataset_test), info = tfds.load(
    'oxford_flowers102',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in oxford_flowers102
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)  # Use efficientnet's preprocess_input
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

test_ds_eff = (dataset_test
               .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
               .batch(batch_size)
               .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define EfficientNetB0 Model
# ---------------------------------------------------
def create_efficientnet(input_shape, num_classes):
    base_model = efficientnet.EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNetB0")
    return model

# ---------------------------------------------------
# 4. Compile and Train the Model
# ---------------------------------------------------
# EfficientNetB0 Model
eff_model = create_efficientnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on oxford_flowers102 ---")
history_eff = eff_model.fit(train_ds_eff, epochs=50, validation_data=test_ds_eff, verbose=1)

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(test_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.4E6Q3R_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.4E6Q3R_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Fine-Tuning EfficientNetB0 on oxford_flowers102 ---
Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.0147 - loss: 4.7369 - val_accuracy: 0.0000e+00 - val_loss: 4.6481
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 31s 94ms/step - accuracy: 0.0296 - loss: 4.5564 - val_accuracy: 0.0049 - val_loss: 4.5493
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.0408 - loss: 4.4390 - val_accuracy: 0.0098 - val_loss: 4.4533
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.0590 - loss: 4.3242 - val_accuracy: 0.0245 - val_loss: 4.3608
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.0962 - loss: 4.2113 - val_accuracy: 0.0490 - val_loss: 4.2708
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.1323 - loss: 4.0971 - val_accuracy: 0.0931 -